In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
# 데이터 로드
train_df = pd.read_csv("./data/train_df.csv", index_col=0)

In [3]:
# Feature와 Label 분리
X = train_df.drop(['BLDS', 'D'], axis=1)
y = train_df['D']

In [4]:
# 데이터 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
# Train, Validation 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# 데이터를 PyTorch Tensor로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

In [7]:
# 모델 정의
class MLPModel(nn.Module):
    def __init__(self, input_size):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [8]:
# 모델 초기화 및 손실 함수, 최적화 기준 정의
model = MLPModel(input_size=X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
# Epochs와 Batch Size 설정
epochs = 20
batch_size = 64

# 주어진 에포크 수만큼 반복
for epoch in range(epochs):
    # 모델을 학습 상태로 설정
    model.train()
    
    # 주어진 배치 크기로 데이터를 나눠 학습
    for i in range(0, len(X_train_tensor), batch_size):
        X_batch = X_train_tensor[i:i+batch_size]
        y_batch = y_train_tensor[i:i+batch_size]

        # 옵티마이저의 그래디언트 초기화
        optimizer.zero_grad()
        
        # 모델을 사용하여 예측 수행
        y_pred = model(X_batch)
        
        # 손실 계산
        loss = criterion(y_pred, y_batch)
        
        # 손실에 대한 역전파 수행하여 그래디언트 계산
        loss.backward()
        
        # 옵티마이저를 사용하여 모델 파라미터 업데이트
        optimizer.step()

    # Validation 데이터로 성능 평가
    model.eval()
    with torch.no_grad():
        # 검증 데이터에 대한 예측 수행
        val_predictions = model(X_val_tensor)
        
        # 이진 분류에서는 0.5를 기준으로 예측값을 0 또는 1로 변환
        val_predictions = (val_predictions > 0.5).float()
        
        # 검증 정확도 계산
        val_accuracy = accuracy_score(y_val_tensor, val_predictions)

    # 현재 에포크의 손실과 검증 정확도 출력
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}, Validation Accuracy: {val_accuracy:.4f}')

RuntimeError: all elements of input should be between 0 and 1

In [ ]:
# 학습된 모델을 사용하여 테스트 데이터 예측
test_df = pd.read_csv("./data/test_df.csv", index_col=0)
X_test = scaler.transform(test_df.drop(['BLDS', 'D'], axis=1))
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_predictions = (test_predictions > 0.5).float()

In [ ]:
# 예측 결과 출력
print(test_predictions.numpy().flatten())

In [ ]:
# 정확도 계산
test_accuracy = accuracy_score(test_df['D'], test_predictions)
print(f"테스트 데이터의 정확도: {test_accuracy:.4f}")